In [1]:
# FedAvg
# FedFair
# data distribution that we used

# Recall
# FedAvg + FairFed + normal model

# Plots withing notbooks
# Plots for Acc vs subGroup
# Recall of positive class
# the data split we use....

In [1]:
import pandas as pd
import time
import numpy as np
import copy
import glob
import sys, os
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from sklearn.metrics import precision_score, recall_score
import pickle
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import os
from torch.utils.data import Dataset
import numpy as np
import torch.nn.functional as F
import pandas as pd
from scipy.stats import multivariate_normal
import torch, random, copy, os
from collections import OrderedDict


In [2]:
# TPR
# Gender: 
# 1: Male, 0: Female
# Fairness: Male- Female
# eq-4: Global- TPR

# Fk = 

In [3]:
# !pip install ray

# data preparation

In [4]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
else:
    # CUDA is not available
    print("CUDA is not available. Running on CPU.")


CUDA is available.


In [5]:
os.getcwd()

'/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB'

In [6]:
 
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        # 9-> we have 9 columns in data 
        self.layer1 = nn.Linear(14, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x



# assigning Models

In [7]:
#create global model and client models
temp_model = DeepNet()
# path="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model_FA.pt"
path="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model_FF.pt"
print("global_model::: FedAVG")
print("global_model_2:: FairFair")
torch.save(temp_model.state_dict(), path)

# this has to be same in follwing code as well
selected_clients=[0,1,2,3]

for client_id in selected_clients:
    client_model_path = f"/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/client_{client_id}_model.pth"
    torch.save(temp_model.state_dict(), client_model_path)
    


global_model::: FedAVG
global_model_2:: FairFair


In [8]:

class Client:
    def __init__(self):
        self.client_id: int = None
        self.valset: DataLoader = None
        self.trainset: DataLoader = None
        self.testset: DataLoader = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")         
        self.model = DeepNet().to(self.device)
        self.criterion = torch.nn.BCELoss()

        self.fainess_score=0

        self.info_collection=[]
        self.client_df = pd.DataFrame()
        self.selected_clients=[0,1,2,3]

        
    def get_client_local_dataset(self):
                
        temp_path_data="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/data_fairFed"
        
        with open(temp_path_data+"/clients_training.pkl", "rb") as f:
            self.trainset = pickle.load(f)
            
        with open(temp_path_data+"/clients_validation.pkl", "rb") as f:
            self.valset = pickle.load(f)
         
        with open(temp_path_data+"/clients_testing_wrong.pkl", "rb") as f:
            self.testset  = pickle.load(f)

        self.trainset = self.trainset[self.client_id]
        self.valset = self.valset[self.client_id]       
        self.testset = self.testset[self.client_id]

    def get_stats(self):
        # 1 male 0 female
        total_pr_Y1_A0 = 0
        total_pr_Y1_A1 = 0

        for client_id in self.selected_clients:
            self.client_id=client_id
            self.get_client_local_dataset()
            for inputs, labels, A in self.trainset:
                pr_Y1_A1 = torch.sum((labels == 1) & (A == 1)).item()
                pr_Y1_A0 = torch.sum((labels == 1) & (A == 0)).item()
          

                total_pr_Y1_A0 += pr_Y1_A0
                total_pr_Y1_A1 += pr_Y1_A1
        temp_sum=total_pr_Y1_A0+total_pr_Y1_A1
        return [total_pr_Y1_A1/temp_sum,total_pr_Y1_A0/temp_sum]
        
    def calculate_fairness(self,y_hat, A, Y):
        # Calculate counts using torch.sum
        y_hat, A, Y = y_hat.to(self.device), A.to(self.device), Y.to(self.device)

        # 1: Male, 0: Female
        predict_Y_male = torch.sum((y_hat[(A == 1) & (Y == 1)] == 1)).item()
        predict_Y_female = torch.sum((y_hat[(A == 0) & (Y == 1)] == 1)).item()
        
        count_Y_male = torch.sum((Y[(A == 1) & (Y == 1)] == 1)).item()
        count_Y_female = torch.sum((Y[(A == 0) & (Y == 1)] == 1)).item()
    
        # Calculate probabilities
        prob_Y_male = predict_Y_male / count_Y_male if count_Y_male > 0 else 0
        prob_Y_female = predict_Y_female / count_Y_female if count_Y_female > 0 else 0
    
        # Calculate Fglobal
        fairness_score = prob_Y_male - prob_Y_female
        
        return  fairness_score

    def calculate_ser_fairness(self,y_hat, A, Y,statistics):
        # print("calculate_ser_fairness statistics",statistics)
        y_hat, A, Y = y_hat.to(self.device), A.to(self.device), Y.to(self.device)

        # for individual client!
        #Male 
        predict_Y_male = torch.sum((y_hat[(A == 1) & (Y == 1)] == 1)).item() #get total
        #Pr(A=0, Y=1)
        count_Y_male = torch.sum((A == 1) & (Y == 1)).item() 
        #Pr(Ŷ=1|A=1, Y=1)  #Female      
        predict_Y_female = torch.sum((y_hat[(A == 0) & (Y == 1)] == 1)).item()  #get total
        #Pr(A=1, Y=1)
        count_Y_female = torch.sum((A == 0) & (Y == 1)).item()
        
        if count_Y_male>0:
            predict_Y_male=predict_Y_male/count_Y_male #Pr(Ŷ=1|A=0, Y=1)
        
        else:
            predict_Y_male=0

        if count_Y_female>0:
            predict_Y_female=predict_Y_female/count_Y_female   #Pr(Ŷ=1|A=1, Y=1) 
        
        else:
            predict_Y_female=0
        
        # print(predict_Y_male*count_Y_male,"::",predict_Y_female*count_Y_female)

        temp_fairness_server=((predict_Y_male * count_Y_male)/statistics[0]) - ((predict_Y_female * count_Y_female)/statistics[1])

        # print("calculate_ser_fairness temp_fairness_server",self.client_id,temp_fairness_server)
        return temp_fairness_server


    def train(self, client_id: int, model_path,statistics, num_epochs=5, learning_rate=0.001):
        self.client_id = client_id
        self.get_client_local_dataset()

        # Define loss function and optimizer
        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.fainess_score=0
      
        self.model.load_state_dict(torch.load(model_path,map_location=self.device))
        for epoch in range(num_epochs):
            # Set the model to training mode
            self.model.train()

            for inputs, labels, sens in self.trainset:
                
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = self.model(inputs).to(self.device)
                loss = self.criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

            # Calculate and print the training accuracy for this epoch (optional)
            
            correct = 0
            total = 0
            size = 0
            loss = 0
            correct = 0
            total = 0
            predicted_labels = []
            true_labels = []
            final_fairness=[]
            final_fairness_server=[]
            
            with torch.no_grad():
                print("======Validation========")
                for inputs, labels,sens in self.valset:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    
                    outputs = self.model(inputs)
                    loss += self.criterion(outputs, labels)
                    predicted = outputs > 0.5

                    predicted_labels.extend(predicted.cpu().numpy())
                    true_labels.extend(labels.cpu().numpy())

                    fairNess_per_batch=self.calculate_fairness(torch.round(outputs).squeeze(), sens.squeeze(), labels.squeeze())
                    final_fairness.append(fairNess_per_batch)
                    self.fainess_score=np.mean(final_fairness)

                    fairNess_per_batch_server=self.calculate_ser_fairness(torch.round(outputs).squeeze(), sens.squeeze(), labels.squeeze(),statistics)
                    final_fairness_server.append(fairNess_per_batch_server)
                    ser_score=np.mean(final_fairness_server)

                    
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            loss = loss / len(self.valset)        
            acc = correct / total
            
            # print("ser_score ",ser_score)
            
            precision = precision_score(true_labels, predicted_labels,zero_division=0.0)            
            recall = recall_score(true_labels, predicted_labels)
            
            # accuracy = 100 * correct / total
            print(f"Epoch {epoch+1}/{num_epochs}, Val Accuracy: {acc:.5f}%")
            print(f"Epoch {epoch+1}/{num_epochs}, Val Fairness: {self.fainess_score:.5f}%")

        # Optionally, save the trained model parameters
        # client_model_path = f"/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/client_{self.client_id}_model.pth"
        #store model
        torch.save(self.model.state_dict(), model_path)

        # Return the trained model parameters
        return list(self.model.state_dict().values()), len(self.trainset.dataset),self.model.state_dict(), self.fainess_score, ser_score

    @torch.no_grad()
    def client_evaluate(self, val=False):
        
        size = 0
        loss = 0
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []

        
        models_directory = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"
                 
        for client_id in self.selected_clients:
            
            print("===============Testing======================")
            # print(" selected client for Testing:: ", client_id)
            model_path = os.path.join(models_directory, f"client_{client_id}_model.pth")            
        
            self.model.load_state_dict(torch.load(model_path,map_location=self.device))

            self.model.eval()

            for inputs, targets,sens in self.testset:
                
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                
                outputs = self.model(inputs)
                loss += self.criterion(outputs, targets)
                predicted = outputs > 0.5
                
                predicted_labels.extend(predicted.cpu().numpy())
                true_labels.extend(targets.cpu().numpy())
                
                total += targets.size(0)
                correct += (predicted == targets).sum().item()                
            
            loss = loss / len(self.testset)
            acc = correct / total
            
            # print("loss: %f\n" % (loss))
            
            
            print(f" Client Accuracy: {acc:.5%}")
    #         print(predicted_labels," :: ",true_labels)
            precision = precision_score(true_labels, predicted_labels,zero_division=0.0)
            
            recall = recall_score(true_labels, predicted_labels)
    
            print(f" Client Precision: {precision:.5%}")
            print(f" Client Recall: {recall:.5%}")
            
            print("\n\n")
            
        return loss, acc,precision,recall

    
    @torch.no_grad()
    def server_evaluate(self):

        # print("Global Model testing Starts")
        # print("kindly check the Path. Select it based on FedAvg Model")
        # print("warning: Test data has already been used")
        temp_path_data="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/"

        with open(temp_path_data+"/testing_client.pkl", "rb") as f:
            testset  = pickle.load(f)
            
        path="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model.pt"
        # path="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model_2.pt"
        
        self.model.load_state_dict(torch.load(path,map_location=self.device))
        self.model.eval()
        size = 0
        loss = 0
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []
        final_fairness=[]

        self.testset=testset[1]
        for inputs, targets,sens in self.testset:
            
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            
            outputs = self.model(inputs)
            loss += self.criterion(outputs, targets)
            predicted = outputs > 0.5

            fairNess_per_batch=self.calculate_fairness(torch.round(outputs).squeeze(), sens.squeeze(), targets.squeeze())
            final_fairness.append(fairNess_per_batch)
            fairness_global=np.mean(final_fairness)
            
#            print("predicted",predicted)
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(targets.cpu().numpy())
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
        
        loss = loss / len(self.testset)
        acc = correct / total
        
        # print("loss: %f\n" % (loss))
        
        print(f"Global Testing Accuracy: {acc:.5%}")
        print(f" Global Fairness: {fairness_global:.5f}%")
        precision = precision_score(true_labels, predicted_labels,zero_division=0.0)
        
        recall = recall_score(true_labels, predicted_labels)
        
        print(f"Global Precision: {precision:.5%}")
        print(f"Global Recall: {recall:.5%}")
        
        return loss, acc, precision, recall,fairness_global
                                            

In [33]:
class Serverbase:
    def __init__ (self,model):
        self.model = model
        self.global_model=model
        self.num_rounds=2
        self.local_epoch=2
        self.optimizer=2
        self.lr=0.001
        self.beta=1
        self.batch_size=32
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

        self.client_id_indices, self.client_num_in_total = [0,1], 2 
        
        self.updated_params_cache = []
        self.weights_cache = []
        self.model_dict = []
        
        self.global_params_dict: OrderedDict[str : torch.Tensor] = None

        self.backbone=DeepNet
        _dummy_model = self.backbone()
        self.global_params_dict = OrderedDict(_dummy_model.state_dict())

        self.temp_dir="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"

        self.fair_global_t_step=0
        self.fair_local_client=0   
        
        self.acc_clients=[]
        self.clients_id=[]
        self.precision_clients=[]
        self.recall_clients=[]
        
        self.global_acc=0
        self.fairness_diff=[]
        self.clients_weights=[]
        self.final_agg_weights_clients=[]
        self.selected_clients=[0,1,2,3]
        self.statistics=0

    def global_fairness(self,weights, all_clients_fairness): 
        # all_clients_fairness is based on equation: 7
        # upper part is done in client class
        # self.statistics cal. pr(y=1,a=0)
        
        # print("global_fairness::",all_clients_fairness)
        # print("weights::",weights)
        
        weight_sum = sum(weights)  
        # print("weights::",weights)
        final_global_score=0
        
        for i in range(len(all_clients_fairness)):
            temp=weights[i]/weight_sum            
            final_global_score+=temp*all_clients_fairness[i]
            
            print("final_global_score:: ",final_global_score)
        self.fair_global=final_global_score

    
        
    def fairFed(self, num_rounds = 2, local_epochs = 2, learning_rate = 0.001, beta = 1, alpha = 0.1, optimizer = 'adam'):
        client = Client()
        #before we start the training, we need statistics
        self.statistics= client.get_stats()        
        
        for round_ in tqdm(range(num_rounds)):
  
            #each round we need empty lists
            self.updated_params_cache = []
            self.weights_cache = []
            self.model_dict=[]
            self.clients_weights=[]
            self.final_agg_weights_clients=[]
            self.fair_global=0
            temp_ser_fairness=[]
            
            
            for client_id in self.selected_clients:
                self.global_acc=0
                
                print("client_id:: ",client_id)
                # weight is length of dataset

                models_directory = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"
                #take a model of particular client
                model_path = os.path.join(models_directory, f"client_{client_id}_model.pth")                       
                
                updated_params_list, weight, model_dict_list,fairness_client,ser_fairness = client.train(client_id, 
                                                                                                         model_path, 
                                                                                                         statistics=self.statistics,
                                                                                                         num_epochs=local_epochs, 
                                                                                                         learning_rate=learning_rate)
                
                # loss_client, acc_client,precision_client,recall_client = client.client_evaluate(val=True)                
                            
                # store it for SecAgg
                self.updated_params_cache.append(updated_params_list)
                self.weights_cache.append(weight)
                self.model_dict.append(model_dict_list)
                temp_ser_fairness.append(ser_fairness)
                
                # print("temp_ser_fairness",temp_ser_fairness)
                #store it for FairFed agg
                
                # self.global_acc=acc_server
                self.fair_local_client=fairness_client
            
            self.global_fairness(self.weights_cache, temp_ser_fairness)

            print("self.fair_global_t_step::",self.fair_global)
            # loss_server, acc_server, precision_server, recall_server,fairness_global = client.server_evaluate()

            # Update the weights of each client
            self.aggregate_client_weight_FedFair_part_1(self.model_dict, self.weights_cache,self.acc_clients, self.global_acc,self.fair_local_client,
                                                 self.fair_global)
            self.client_weights_update_part_2(self.clients_weights)

            # agg all the weights on serverside
            self.aggregate_parameters_FedAvg_updated(self.final_agg_weights_clients)
        
            
            torch.save(self.global_model.state_dict(), os.path.join(self.temp_dir, "global_model_FF.pt"),)

        # Test Clients Models and Global Model
        # loss_client, acc_client = client.client_evaluate()        
        # loss_client, accuracy_server, pre_server, recall_ser,fairness_global = client.server_evaluate()
            

    @torch.no_grad()
    def aggregate_parameters_FedAvg(self, updated_params_cache, weights_cache):
        weight_sum = sum(weights_cache)  
        
        weights = torch.tensor(weights_cache, device=self.device) / weight_sum
        
        aggregated_params = []

        for params in zip(*updated_params_cache):
            aggregated_params.append(
                torch.sum(weights * torch.stack(params, dim=-1), dim=-1)
            )

        self.global_params_dict = OrderedDict(
            zip(self.global_params_dict.keys(), aggregated_params)
        )

    @torch.no_grad()
    def aggregate_parameters_FedAvg_updated(self, model_dict_list):        
        w_avg = copy.deepcopy(model_dict_list[0])
        for k in w_avg.keys():
            for i in range(1, len(model_dict_list)):
                w_avg[k] += model_dict_list[i][k]
            w_avg[k] = torch.div(w_avg[k], len(model_dict_list))
    
        self.global_model.load_state_dict(w_avg)
        # torch.save(self.global_model.state_dict(), os.path.join(self.temp_dir, "global_model_FA.pt"),)        
        # torch.save(self.global_model.state_dict(), os.path.join(self.temp_dir, "global_model_FF.pt"),)

                
        
    @torch.no_grad()
    def aggregate_client_weight_FedFair_part_1 (self, model_dict_list ,weights_cache, acc_clients, global_acc,fairness_client,fairness_global):
        
        if not self.fair_local_client:
            
            sum_acc_client=sum(acc_clients) /len(acc_clients)
            Delta_t_C_i= abs(sum_acc_client-global_acc)
            
        else:            
            Delta_t_C_i = abs(self.fair_global - self.fair_local_client) 
        
        for client_id in range(len(self.selected_clients)):
        
            w_avg = copy.deepcopy(model_dict_list[client_id])
            temp = copy.deepcopy(model_dict_list[client_id])
            
            for k in w_avg.keys():
                for i in range(1, len(model_dict_list)):
                    w_avg[k] += model_dict_list[i][k]
                w_avg[k] = torch.div(w_avg[k], len(model_dict_list))
                
                w_avg[k]=temp[k]-(self.beta*(Delta_t_C_i-w_avg[k]))
                
            self.clients_weights.append(w_avg)

            
    def client_weights_update_part_2(self,agg_weights):
        sum_params = OrderedDict()
        final_agg_weights_clients=[]
        for model in agg_weights:
            for name, param in model.items():
                if name in sum_params:
                    sum_params[name] += param
                else:
                    sum_params[name] = param

            updated_client_weights = OrderedDict()
            
            for name, param in model.items():
                updated_param = torch.div(param, sum_params[name])
                updated_client_weights[name] = updated_param

                self.final_agg_weights_clients.append(updated_client_weights)
        
model = DeepNet()
server = Serverbase(model)
server.fairFed(num_rounds=3,local_epochs=15)    

  0%|                                                                                                                        | 0/5 [00:00<?, ?it/s]

client_id::  0
======Validation========
Epoch 1/1, Val Accuracy: 0.78052%
Epoch 1/1, Val Fairness: 0.71191%
client_id::  1
======Validation========
Epoch 1/1, Val Accuracy: 0.78089%
Epoch 1/1, Val Fairness: -0.68470%
client_id::  2
======Validation========
Epoch 1/1, Val Accuracy: 0.81195%
Epoch 1/1, Val Fairness: 0.75273%
client_id::  3
======Validation========


 20%|██████████████████████▍                                                                                         | 1/5 [00:07<00:30,  7.50s/it]

Epoch 1/1, Val Accuracy: 0.77519%
Epoch 1/1, Val Fairness: -0.79279%
final_global_score::  6.690567043286983
final_global_score::  -0.6720699260581267
final_global_score::  12.424069503271616
final_global_score::  0.1313011118729257
self.fair_global_t_step:: 0.1313011118729257
client_id::  0
======Validation========
Epoch 1/1, Val Accuracy: 0.78983%
Epoch 1/1, Val Fairness: 0.68530%
client_id::  1
======Validation========
Epoch 1/1, Val Accuracy: 0.78107%
Epoch 1/1, Val Fairness: -0.76805%
client_id::  2
======Validation========
Epoch 1/1, Val Accuracy: 0.81559%
Epoch 1/1, Val Fairness: 0.82304%
client_id::  3
======Validation========


 40%|████████████████████████████████████████████▊                                                                   | 2/5 [00:15<00:23,  7.95s/it]

...

Epoch 1/1, Val Accuracy: 0.77409%
Epoch 1/1, Val Fairness: -0.78923%
final_global_score::  6.233316978270512
final_global_score::  -1.9445948981381012
final_global_score::  10.790700784445264
final_global_score::  -1.48673450311526
self.fair_global_t_step:: -1.48673450311526
client_id::  0
======Validation========
Epoch 1/1, Val Accuracy: 0.78658%
Epoch 1/1, Val Fairness: 0.67211%
client_id::  1
======Validation========
Epoch 1/1, Val Accuracy: 0.78562%
Epoch 1/1, Val Fairness: -0.71150%
client_id::  2
======Validation========
Epoch 1/1, Val Accuracy: 0.81443%
Epoch 1/1, Val Fairness: 0.76815%
client_id::  3
======Validation========


 80%|█████████████████████████████████████████████████████████████████████████████████████████▌                      | 4/5 [00:36<00:09,  9.61s/it]

Epoch 1/1, Val Accuracy: 0.77409%
Epoch 1/1, Val Fairness: -0.81708%
final_global_score::  6.270107213386781
final_global_score::  -1.3995159863958202
final_global_score::  11.926816867582474
final_global_score::  -0.7251842423104016
self.fair_global_t_step:: -0.7251842423104016
client_id::  0
======Validation========
Epoch 1/1, Val Accuracy: 0.79329%
Epoch 1/1, Val Fairness: 0.70236%
client_id::  1
======Validation========
Epoch 1/1, Val Accuracy: 0.78508%
Epoch 1/1, Val Fairness: -0.74247%
client_id::  2
======Validation========
Epoch 1/1, Val Accuracy: 0.81686%
Epoch 1/1, Val Fairness: 0.77758%
client_id::  3
======Validation========


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:48<00:00,  9.75s/it]

Epoch 1/1, Val Accuracy: 0.77563%
Epoch 1/1, Val Fairness: -0.78309%
final_global_score::  6.574940590064427
final_global_score::  -1.4419293293933713
final_global_score::  12.077268285776949
final_global_score::  -0.10597793534912014
self.fair_global_t_step:: -0.10597793534912014


In [30]:
# 136404